In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna, joblib

In [17]:
sc = StandardScaler()

In [18]:
### create an objective function 
def objective(
    trial #this is the ONLY arguement is default and a must
    ):
    
    #get data
    df_train = pd.read_csv('train_data.csv')
    df_test = pd.read_csv('test_data.csv')
    df_train.columns = ['Aerosol', 
              'Blue', 
              'Green', 
              'Red', 
              'NIR', 
              'SWIR-1', 
              'SWIR-2',
              'Chl-a']
              
    df_test.columns = ['Aerosol', 
              'Blue', 
              'Green', 
              'Red', 
              'NIR', 
              'SWIR-1', 
              'SWIR-2',
              'Chl-a']
    
    #separate X and y
    # and compute NDVI
    X_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]
    X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]
    
    #add NDVI as features
    X_train['NDVI'] = (X_train['NIR'] - X_train['Red']) / (X_train['NIR'] + X_train['Red']) 
    X_test['NDVI'] = (X_test['NIR'] - X_test['Red']) / (X_test['NIR'] + X_test['Red']) 
    
    X_train_sc = sc.fit_transform(X_train)
    X_test_sc = sc.transform(X_test)
    
    dtrain = lgb.Dataset(X_train_sc, label = y_train)
    dvalid = lgb.Dataset(X_test_sc, label = y_test)
    
    # in optuna, setting of parameters is as follows:
    '''
    dictionary = {
    hyperparameter_alias: trial.suggest_distribution(
                           'hyperparameter_alias',
                           ['distribution values]'
                           )
    }    
    
    distribution to choose from:
    
    uniform — float values
    loguniform — float values
    discrete_uniform — float values with intervals
    int — integer values
    categorical — categorical values from a list
    '''
    _EPS = 1e-12
    #example:
    params = {
        'task': 'train',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'rf']),
        'metric': 'rmse',
        'objective': trial.suggest_categorical('objective', ['regression_l1', 'regression_l2']),
        'verbosity': 0,
        "seed": 42,
        "learning_rate": trial.suggest_loguniform('learning_rate', 0.05, 1),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': min(trial.suggest_float("feature_fraction", 0.4, 1.0 + 1e-12), 1.0),
        'bagging_fraction': min(trial.suggest_float("bagging_fraction", 0.4, 1.0 + 1e-12), 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        }          
    
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse", 'eval')

    model = lgb.train(
            params, 
            dtrain, 
            valid_names = ['eval', 'train'], 
            valid_sets = [dvalid, dtrain],
            early_stopping_rounds = 500,
            num_boost_round  = 10000,
            )

    joblib.dump(model, 'OPTIMIZED_MODEL.sav') #save model for future use

    prediction = model.predict(X_test_sc) #apply to test
    
    rmse = np.sqrt(mean_squared_error(y_test, prediction))
    return rmse

In [28]:
#optimize hyper param
study = optuna.create_study(direction = 'minimize') # minimize for regression, maximize for classification
study.optimize(objective, n_trials = 30)

[I 2021-03-03 17:19:07,368] A new study created in memory with name: no-name-8724e0bb-7629-4527-abef-0ca4ddafd628


[1]	train's rmse: 2.16969	eval's rmse: 2.60823
Training until validation scores don't improve for 500 rounds
[2]	train's rmse: 2.16969	eval's rmse: 2.60823
[3]	train's rmse: 2.16969	eval's rmse: 2.60823
[4]	train's rmse: 2.16969	eval's rmse: 2.60823
[5]	train's rmse: 2.07459	eval's rmse: 2.48474
[6]	train's rmse: 2.04167	eval's rmse: 2.42885
[7]	train's rmse: 2.03397	eval's rmse: 2.40292
[8]	train's rmse: 2.03698	eval's rmse: 2.39139
[9]	train's rmse: 1.98904	eval's rmse: 2.33033
[10]	train's rmse: 1.96283	eval's rmse: 2.29392
[11]	train's rmse: 1.94976	eval's rmse: 2.27276
[12]	train's rmse: 1.94474	eval's rmse: 2.26124
[13]	train's rmse: 1.92544	eval's rmse: 2.24435
[14]	train's rmse: 1.91393	eval's rmse: 2.23538
[15]	train's rmse: 1.90779	eval's rmse: 2.23175
[16]	train's rmse: 1.90536	eval's rmse: 2.23176
[17]	train's rmse: 1.89952	eval's rmse: 2.22582
[18]	train's rmse: 1.89696	eval's rmse: 2.22302
[19]	train's rmse: 1.89678	eval's rmse: 2.2225
[20]	train's rmse: 1.89832	eval's rm

[I 2021-03-03 17:19:09,921] Trial 0 finished with value: 2.156299000455626 and parameters: {'boosting_type': 'rf', 'objective': 'regression_l2', 'learning_rate': 0.7276693042893765, 'lambda_l1': 3.3529176656274756e-05, 'lambda_l2': 0.0011762597260586286, 'num_leaves': 108, 'feature_fraction': 0.9657941456050193, 'bagging_fraction': 0.743053336476548, 'bagging_freq': 4, 'min_child_samples': 13}. Best is trial 0 with value: 2.156299000455626.



[521]	train's rmse: 1.87379	eval's rmse: 2.18972
[522]	train's rmse: 1.87395	eval's rmse: 2.19023
[523]	train's rmse: 1.87412	eval's rmse: 2.19074
[524]	train's rmse: 1.87429	eval's rmse: 2.19125
[525]	train's rmse: 1.87447	eval's rmse: 2.19128
[526]	train's rmse: 1.87466	eval's rmse: 2.19131
[527]	train's rmse: 1.87484	eval's rmse: 2.19135
[528]	train's rmse: 1.87503	eval's rmse: 2.19139
[529]	train's rmse: 1.87497	eval's rmse: 2.19139
Early stopping, best iteration is:
[29]	train's rmse: 1.87072	eval's rmse: 2.1563
[1]	train's rmse: 4.05475	eval's rmse: 3.97313
Training until validation scores don't improve for 500 rounds
[2]	train's rmse: 3.56808	eval's rmse: 3.52964
[3]	train's rmse: 3.18426	eval's rmse: 3.19049
[4]	train's rmse: 2.83096	eval's rmse: 2.8829
[5]	train's rmse: 2.58138	eval's rmse: 2.668
[6]	train's rmse: 2.39737	eval's rmse: 2.5227
[7]	train's rmse: 2.21509	eval's rmse: 2.38765
[8]	train's rmse: 2.07694	eval's rmse: 2.29277
[9]	train's rmse: 1.95879	eval's rmse: 2.2

[I 2021-03-03 17:19:12,155] Trial 1 finished with value: 1.9947815643578437 and parameters: {'boosting_type': 'gbdt', 'objective': 'regression_l2', 'learning_rate': 0.17568147026536265, 'lambda_l1': 5.3812219108034475e-06, 'lambda_l2': 4.663884306269143e-08, 'num_leaves': 50, 'feature_fraction': 0.5088299735497699, 'bagging_fraction': 0.7456699147282794, 'bagging_freq': 2, 'min_child_samples': 15}. Best is trial 1 with value: 1.9947815643578437.


...

[1]	train's rmse: 4.46087	eval's rmse: 4.34473
Training until validation scores don't improve for 500 rounds
[2]	train's rmse: 4.24634	eval's rmse: 4.13754
[3]	train's rmse: 4.05318	eval's rmse: 3.9544
[4]	train's rmse: 3.88174	eval's rmse: 3.79377
[5]	train's rmse: 3.72136	eval's rmse: 3.644
[6]	train's rmse: 3.56719	eval's rmse: 3.49845
[7]	train's rmse: 3.42828	eval's rmse: 3.37516
[8]	train's rmse: 3.29636	eval's rmse: 3.25561
[9]	train's rmse: 3.17548	eval's rmse: 3.14997
[10]	train's rmse: 3.05944	eval's rmse: 3.05381
[11]	train's rmse: 2.95623	eval's rmse: 2.969
[12]	train's rmse: 2.8604	eval's rmse: 2.89155
[13]	train's rmse: 2.77421	eval's rmse: 2.81927
[14]	train's rmse: 2.69705	eval's rmse: 2.75582
[15]	train's rmse: 2.62767	eval's rmse: 2.69796
[16]	train's rmse: 2.56479	eval's rmse: 2.64852
[17]	train's rmse: 2.50661	eval's rmse: 2.60002
[18]	train's rmse: 2.45782	eval's rmse: 2.56284
[19]	train's rmse: 2.40767	eval's rmse: 2.53142
[20]	train's rmse: 2.36127	eval's rmse: 2

[I 2021-03-03 17:20:31,235] Trial 28 finished with value: 1.9777134884898724 and parameters: {'boosting_type': 'gbdt', 'objective': 'regression_l2', 'learning_rate': 0.06650012235541161, 'lambda_l1': 9.696298021406411, 'lambda_l2': 1.020536375223314e-06, 'num_leaves': 26, 'feature_fraction': 0.886328521698448, 'bagging_fraction': 0.8145880166135706, 'bagging_freq': 7, 'min_child_samples': 7}. Best is trial 23 with value: 1.914181570898599.


[1]	train's rmse: 4.1438	eval's rmse: 4.07434
Training until validation scores don't improve for 500 rounds
[2]	train's rmse: 3.69522	eval's rmse: 3.68218
[3]	train's rmse: 3.30578	eval's rmse: 3.33578
[4]	train's rmse: 2.97639	eval's rmse: 3.06671
[5]	train's rmse: 2.69957	eval's rmse: 2.84522
[6]	train's rmse: 2.4556	eval's rmse: 2.66047
[7]	train's rmse: 2.23155	eval's rmse: 2.49644
[8]	train's rmse: 2.04347	eval's rmse: 2.36215
[9]	train's rmse: 1.88726	eval's rmse: 2.26962
[10]	train's rmse: 1.74868	eval's rmse: 2.18526
[11]	train's rmse: 1.63275	eval's rmse: 2.13336
[12]	train's rmse: 1.53937	eval's rmse: 2.10248
[13]	train's rmse: 1.45459	eval's rmse: 2.06784
[14]	train's rmse: 1.37852	eval's rmse: 2.03631
[15]	train's rmse: 1.31476	eval's rmse: 2.01323
[16]	train's rmse: 1.25887	eval's rmse: 1.99237
[17]	train's rmse: 1.21257	eval's rmse: 1.97779
[18]	train's rmse: 1.17013	eval's rmse: 1.97196
[19]	train's rmse: 1.11449	eval's rmse: 1.96159
[20]	train's rmse: 1.06669	eval's rms

[I 2021-03-03 17:20:33,319] Trial 29 finished with value: 1.9238289144636769 and parameters: {'boosting_type': 'gbdt', 'objective': 'regression_l2', 'learning_rate': 0.13604398840185541, 'lambda_l1': 0.15044842982607262, 'lambda_l2': 0.0010771980704158725, 'num_leaves': 104, 'feature_fraction': 0.7656775620185373, 'bagging_fraction': 0.7920969697453408, 'bagging_freq': 6, 'min_child_samples': 6}. Best is trial 23 with value: 1.914181570898599.



[489]	train's rmse: 0.0502476	eval's rmse: 1.96005
[490]	train's rmse: 0.0502351	eval's rmse: 1.96004
[491]	train's rmse: 0.0502259	eval's rmse: 1.96002
[492]	train's rmse: 0.0502169	eval's rmse: 1.96002
[493]	train's rmse: 0.0502569	eval's rmse: 1.95997
[494]	train's rmse: 0.0503348	eval's rmse: 1.95991
[495]	train's rmse: 0.0504173	eval's rmse: 1.95985
[496]	train's rmse: 0.0505408	eval's rmse: 1.95981
[497]	train's rmse: 0.0506227	eval's rmse: 1.9598
[498]	train's rmse: 0.050713	eval's rmse: 1.9598
[499]	train's rmse: 0.0505693	eval's rmse: 1.95979
[500]	train's rmse: 0.0504621	eval's rmse: 1.95979
[501]	train's rmse: 0.0503762	eval's rmse: 1.95977
[502]	train's rmse: 0.0503046	eval's rmse: 1.95977
[503]	train's rmse: 0.0502486	eval's rmse: 1.95979
[504]	train's rmse: 0.0501987	eval's rmse: 1.95977
[505]	train's rmse: 0.05014	eval's rmse: 1.9598
[506]	train's rmse: 0.0500968	eval's rmse: 1.95982
[507]	train's rmse: 0.0500683	eval's rmse: 1.95985
[508]	train's rmse: 0.0500396	eval's

In [29]:
study.best_trial #contains the hyperparameter settings

FrozenTrial(number=23, value=1.914181570898599, datetime_start=datetime.datetime(2021, 3, 3, 17, 20, 12, 88625), datetime_complete=datetime.datetime(2021, 3, 3, 17, 20, 15, 808867), params={'boosting_type': 'gbdt', 'objective': 'regression_l2', 'learning_rate': 0.051013418735316535, 'lambda_l1': 4.794388279256722, 'lambda_l2': 7.14288739917065e-07, 'num_leaves': 84, 'feature_fraction': 0.7801327027938199, 'bagging_fraction': 0.9077382376670677, 'bagging_freq': 7, 'min_child_samples': 6}, distributions={'boosting_type': CategoricalDistribution(choices=('gbdt', 'rf')), 'objective': CategoricalDistribution(choices=('regression_l1', 'regression_l2')), 'learning_rate': LogUniformDistribution(high=1, low=0.05), 'lambda_l1': LogUniformDistribution(high=10.0, low=1e-08), 'lambda_l2': LogUniformDistribution(high=10.0, low=1e-08), 'num_leaves': IntUniformDistribution(high=256, low=2, step=1), 'feature_fraction': UniformDistribution(high=1.000000000001, low=0.4), 'bagging_fraction': UniformDistri

In [21]:
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    "{}": {},'.format(key, value))

Best trial:
  Value: 2.269627402687017
  Params: 
    "boosting_type": gbdt,
    "objective": regression_l2,
    "learning_rate": 0.32875122929920914,
    "lambda_l1": 4.302884701416492e-05,
    "lambda_l2": 0.00446641166459144,
    "num_leaves": 7,
    "feature_fraction": 0.902653559382219,
    "bagging_fraction": 0.20289579693458237,
    "bagging_freq": 4,
    "min_child_samples": 36,


In [22]:
model_tuned = joblib.load('OPTIMIZED_MODEL.sav') #call saved model

In [23]:
df_test = pd.read_csv('test_data.csv')
df_test.columns = ['Aerosol', 
          'Blue', 
          'Green', 
          'Red', 
          'NIR', 
          'SWIR-1', 
          'SWIR-2',
          'Chl-a']

In [24]:
X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]
X_test['NDVI'] = (X_test['NIR'] - X_test['Red']) / (X_test['NIR'] + X_test['Red']) 

In [25]:
X_test_sc = sc.fit_transform(X_test) #standardize

In [26]:
pred_test = model_tuned.predict(X_test_sc)
rmse = np.sqrt(mean_squared_error(y_test, pred_test))
rmse

3.2018627584470645

In [27]:
r2_score(y_test, pred_test)

0.5003501807833579